In [ ]:
#@title Download & auth ngrok
ngrok_token = "" #@param {type:"string"}
!mkdir -p /drive/ngrok
%cd /drive/ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok/ngrok /ngrok
!chmod +x /ngrok

!/ngrok authtoken $ngrok_token
print("\nLogged in with " + ngrok_token + " token")

In [ ]:
#@title Save ngrok run config
%%writefile /drive/ngrok/tcp25565.yml
tunnels:
 tcp25565:
    proto: tcp
    addr: 25565
    region: eu

In [ ]:
#@title Save ngrok run script
%%writefile /drive/ngrok/run_ngrok.sh
#!/bin/sh
set -x
/ngrok start --config ~/.ngrok2/ngrok.yml --log=stdout --config /drive/ngrok/tcp25565.yml "$@"

In [ ]:
#@title start ngrok
import os
import requests
import urllib.parse

get_ipython().system_raw('bash /drive/ngrok-ssh/run_ngrok.sh tcp25565 &')

def get_ngrok_info():
  return requests.get('http://localhost:4040/api/tunnels').json()

def get_ngrok_tunnels():
  for tunnel in get_ngrok_info()['tunnels']:
    name = tunnel['name']
    yield name, tunnel

def get_ngrok_tunnel(name):
  for name1, tunnel in get_ngrok_tunnels():
    if name == name1:
      return tunnel

def get_ngrok_url(name, local=False):
  if local:
    return get_ngrok_tunnel(name)['config']['addr']
  else:
    return get_ngrok_tunnel(name)['public_url']
    
print(get_ngrok_url('tcp25565'))

In [ ]:
#@title Downloads newer java version
java_version =  15#@param {type:"integer"}

!apt-get install -y wget apt-transport-https gnupg
!wget https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public
!gpg --no-default-keyring --keyring ./adoptopenjdk-keyring.gpg --import public
!gpg --no-default-keyring --keyring ./adoptopenjdk-keyring.gpg --export --output adoptopenjdk-archive-keyring.gpg

!rm adoptopenjdk-keyring.gpg
!mv adoptopenjdk-archive-keyring.gpg /usr/share/keyrings 
!echo "deb [signed-by=/usr/share/keyrings/adoptopenjdk-archive-keyring.gpg] https://adoptopenjdk.jfrog.io/adoptopenjdk/deb bionic main" | sudo tee /etc/apt/sources.list.d/adoptopenjdk.list
!apt-get update
!apt-get install adoptopenjdk-$java_version-hotspot

In [ ]:
#@title minecraft server
server_directory = "/content/drive/MyDrive/minecraft-server/server2" #@param {type:"string"}
start_script = "start.sh" #@param {type:"string"}
%cd $server_directory
!bash $start_script